In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from config import api_key

In [13]:
company_code = "TSLA"
time_function = "TIME_SERIES_MONTHLY"
#https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo
test_url = f"https://www.alphavantage.co/query?function={time_function}&symbol={company_code}&apikey={api_key}"
response = requests.get(test_url).json()
pprint(response)

{'Meta Data': {'1. Information': 'Monthly Prices (open, high, low, close) and '
                                 'Volumes',
               '2. Symbol': 'TSLA',
               '3. Last Refreshed': '2024-09-12',
               '4. Time Zone': 'US/Eastern'},
 'Monthly Time Series': {'2010-07-30': {'1. open': '25.0000',
                                        '2. high': '25.9200',
                                        '3. low': '14.9800',
                                        '4. close': '19.9400',
                                        '5. volume': '64575800'},
                         '2010-08-31': {'1. open': '20.5000',
                                        '2. high': '22.1800',
                                        '3. low': '17.3900',
                                        '4. close': '19.4800',
                                        '5. volume': '15038200'},
                         '2010-09-30': {'1. open': '19.6200',
                                        '2. high': '23